# Limpieza de datos del SNIIM (Sistema Nacional de información e Integración de Mercados)

Una vez descargadas todas las tablas del SNIIM en formato CSV, se unifican en un sólo dataframe que contenga toda la información necesaria.

Por ejemplo, un dataframe que unifique a todas las frutas y hortalizas reportadas en el periodo del 01/01/2020 al 31/12/2023 tendrá un total de 2,086,593 de observaciones y un total de 9 columnas.

#### Importar librerías

In [7]:
import pandas as pd
import os
import re
import numpy as np

#### Creación del dataframe con todos los archivos CSV descargados

In [85]:
# Se crea un dataframe que contiene la información de todos los archivos CSV
path = r"./raw_data/"
archivos = next(os.walk(path))[2]
#print("Archivos = ", archivos)

lista_df = []

for i in range(len(archivos)):
    temp_df = pd.read_csv(path + archivos[i], thousands=',') # uno de los archivos utiliza commas en sus números
    nombre_producto = re.search(r"Precio_de_(.+)_\d{4}-\d{4}.csv", archivos[i]).group(1).replace("_", " ")
    temp_df['Producto'] = nombre_producto
    #print(nombre_producto)
    lista_df.append(temp_df)

df_sniim_fyh = pd.concat(lista_df, ignore_index=True)
df_sniim_fyh.rename(columns={"Obs.": "Observaciones"}, inplace=True)
df_sniim_fyh

,Fecha,Presentación,Origen,Destino,Precio Mín,Precio Max,Precio Frec,Observaciones,Producto
0,30/03/2020,Kilogramo,Veracruz,Guanajuato: Central de Abasto de León,13.0,14.0,13.0,NaN,Toronja Amarilla
1,31/03/2020,Kilogramo,Veracruz,Guanajuato: Central de Abasto de León,13.0,14.0,13.0,NaN,Toronja Amarilla
2,01/04/2020,Kilogramo,Veracruz,Guanajuato: Central de Abasto de León,13.0,14.0,13.0,NaN,Toronja Amarilla
3,02/04/2020,Kilogramo,Veracruz,Guanajuato: Central de Abasto de León,13.0,14.0,13.0,NaN,Toronja Amarilla
4,03/04/2020,Kilogramo,Veracruz,Guanajuato: Central de Abasto de León,13.0,14.0,13.0,NaN,Toronja Amarilla
...,...,...,...,...,...,...,...,...,...
2086588,26/09/2023,Kilogramo,Colima,Colima: Centros de distribución de Colima,15.0,18.0,18.0,NaN,Sandía Jubilee
2086589,29/09/2023,Kilogramo,Colima,Colima: Centros de distribución de Colima,15.0,18.0,18.0,NaN,Sandía Jubilee
2086590,03/10/2023,Kilogramo,Colima,Colima: Centros de distribución de Colima,15.0,18.0,18.0,NaN,Sandía Jubilee
2086591,06/10/2023,Kilogramo,Colima,Colima: Centros de distribución de Colima,15.0,20.0,20.0,NaN,Sandía Jubilee


#### Análisis básico del dataframe y algunas características

La variable "Observaciones" es la única que presenta valores perdidos, más del 95% de las observaciones en esta variable son valores perdidos. Se entiende que es una variable opcional que sólo indica algún comentario.

In [90]:
df_sniim_fyh.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086593 entries, 0 to 2086592
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Fecha          2086593 non-null  object 
 1   Presentación   2086593 non-null  object 
 2   Origen         2086593 non-null  object 
 3   Destino        2086593 non-null  object 
 4   Precio Mín     2086593 non-null  float64
 5   Precio Max     2086593 non-null  float64
 6   Precio Frec    2086593 non-null  float64
 7   Observaciones  90867 non-null    object 
 8   Producto       2086593 non-null  object 
dtypes: float64(3), object(6)
memory usage: 143.3+ MB


In [93]:
df_sniim_fyh.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,Precio Mín,Precio Max,Precio Frec
count,2086593.00,2086593.00,2086593.00
mean,184.94,205.86,194.54
std,259.10,286.29,269.42
min,1.00,1.00,0.00
25%,19.00,21.00,20.00
50%,65.00,72.00,70.00
75%,255.00,290.00,275.00
max,11000.00,6650.00,4200.00


In [94]:
df_sniim_fyh['Producto'].value_counts()

Tomate Saladette            44917
Jitomate Saladette          44917
Manzana Golden Delicious    37192
Papa Alpha                  36522
Manzana Red Delicious       35416
                            ...  
Melón Cantaloupe # 18          20
Mango Mangloba                 10
Pistache                        5
Melón Cantaloupe # 17           2
Melón Gotas de miel             1
Name: Producto, Length: 221, dtype: int64

No se encuentran observaciones duplicadas:

In [92]:
df_sniim_fyh[df_sniim_fyh.duplicated()]

,Fecha,Presentación,Origen,Destino,Precio Mín,Precio Max,Precio Frec,Observaciones,Producto
